# Script to convert JSON-Schema to something Jekyll can render
- GitHub Issue : https://github.com/BioSchemas/specifications/issues/568
- Meetings Notes : https://docs.google.com/document/d/1zlP871DzT68_2E-jRaWXJriWmvI5puXPbR5iRZ-lA34/edit
- Slides : https://docs.google.com/presentation/d/1DD31yAZ6YC0xESEg-XsuvgmP9rk0YPtuWY3beSXBnBU/edit?usp=sharing

#### Import Libraries

In [ ]:
import json
import yaml

In [ ]:
import colorama
from colorama import Fore
from colorama import Style

# Mapping Functions

## Generate Profile [Spec_info + Mapping (sum of properties)]

Funtion that takes a profile graph and generate a transformed profile (the one that will be inserted in our YAML file)

In [ ]:
def generate_transformed_profile(g):
    
    transformed_profile = dict()
    
    ###Spec_info
    transformed_profile["spec_info"] = generate_spec_info(g)    
    
    ### Mapping
    transformed_profile["mapping"] = list(dict())
            
    # Browse properties by marginality and add them to the mappings        
    if "$validation" in g.keys():
        for req_label in g['$validation']["required"]:
            prop = g['$validation']["properties"][req_label]
            new_p = generate_property (g, prop, req_label, "Required")
            transformed_profile["mapping"].append(new_p)

        for reco_label in g['$validation']["recommended"]:
            prop = g['$validation']["properties"][reco_label]
            new_p = generate_property (g, prop, reco_label, "Minimum")
            transformed_profile["mapping"].append(new_p)

        for opt_label in g['$validation']["optional"]:
            prop = g['$validation']["properties"][opt_label]
            new_p = generate_property (g, prop, opt_label, "Optional")
            transformed_profile["mapping"].append(new_p)

    return transformed_profile

## Generate the spec_info

In [ ]:
def generate_spec_info(g):
    spec_info = dict()    
        
    if "rdfs:label" in g.keys():
        spec_info["title"] = g['rdfs:label']
    else:
        raise Exception("Please Provide the title of your profile!")

    if "rdfs:comment" in g.keys():
        spec_info["description"] = g['rdfs:comment']
    else:
        spec_info["description"]
        
    if "schema:schemaVersion" in g.keys():
        spec_info["version"] = g['schema:schemaVersion'][0].split('/')[-1]
    else:
        raise Exception("Please Provide the version of your profile!")
        
    if "rdfs:subClassOf" in g.keys():
        spec_info["official_type"] = g['rdfs:subClassOf']
    else:
        spec_info["official_type"]=""

    spec_info["full_example"] = "https://github.com/BioSchemas/specifications/tree/master/"+spec_info["title"]+"/examples"

    return spec_info

## Generate Property

Function that takes a JSON_LD property and transforme it

In [ ]:
def generate_property (g, prop, req_label, marginality):
    
    print(Fore.GREEN + Style.BRIGHT + f'Property : {req_label}' + Style.RESET_ALL)
    new_p = dict()
    #If I want to create a dict, new_p[] = {'description':prop["description"]}
    
    new_p['property'] = req_label
    new_p['marginality'] = marginality
    new_p['cardinality'], new_p['expected_types'] = generate_types_cardianlity(g, prop)

    if "description" in prop.keys():
        new_p['description'] = prop["description"]
    else:
        new_p['description']=""
    if "type_url" in prop.keys():
        new_p['type_url'] = prop["type_url"]
    else:
        new_p['type_url']=""
    if "bsc_description" in prop.keys():
        new_p['bsc_description'] = prop["bsc_description"]
    else:
        new_p['bsc_description']=""
    if "controlled_vocab" in prop.keys():
        new_p['controlled_vocab'] = prop["controlled_vocab"]
    else:
        new_p['controlled_vocab']=""
    
    if "example" in prop.keys():
        new_p['example'] = prop["example"]
    else:
        new_p['example']=""

    # Here we'll suppose that all properties are in default schemas.org 
    new_p['type']=""

    return new_p

## Expected Types & Cardinality

1. Extract the union of types in the anyOf or oneOf fild of the property
2. Cardinality = MANY or ONE

I don't need this [flat-list](https://stackoverflow.com/questions/27612626/how-to-convert-a-dictionary-into-a-flat-list), bcause I don't need the keys in my list :D 

In [ ]:
def generate_types_cardianlity(g,prop):
    
    list_types=list()
    cardianliy="ONE"
    
    if "type" in prop.keys():
        if "format" in prop.keys():
            list_types.append(prop["format"])
        else :
            list_types.append(prop["type"])
    
    if "anyOf" in prop.keys():
        for e in prop["anyOf"]:
            if "format" in e.keys():
                list_types.append(e["format"])
            elif "items" in e.keys():
                list_types.append(e["items"])
                cardianliy = "MANY"
            else :
                for t in e.keys():
                    list_types.append(e[t])
    
    if "oneOf" in prop.keys():
        for e in prop["oneOf"]:
            if "format" in e.keys():
                list_types.append(e["format"])
            elif "items" in e.keys():
                list_types.append(e["items"])
                cardianliy = "MANY"
            else :
                for t in e.keys():
                    list_types.append(e[t])

    i=1
    j=1
    while i==1:
        i=0
        j=j+1
        for t in list_types:
            if isinstance(t,dict):
                i=1
                e=t
                list_types.remove(t)
                if "format" in e.keys():
                    list_types.append(e["format"])
                else:
                    for sous_type in e.keys():
                        list_types.append(e[sous_type])
                   
    if len(list_types)==0:
        cardianliy=""
        
    #Some cleaning up
    expected_types =[]
    
    for t in list_types:
        if len(t.split('/'))>1 :
            expected_types.append(t.split('/')[-1].capitalize())
            ##If we ever needed it
            #external_type= g['$validation']["definitions"][t.split('/')[-1]]
            #print(Fore.YELLOW + f'Def of the External Type : {external_type}' + Style.RESET_ALL)            

        else :
            expected_types.append(t.capitalize())    

    ##Remove duplicates
    clean_expected_types = list(set(expected_types))

    #Replace 'String' with 'Text'
    for i in clean_expected_types:
        if i == "String":
            clean_expected_types.remove(i)
            clean_expected_types.append("Text")
    print(Fore.MAGENTA + f'Expected Types : {clean_expected_types}' + Style.RESET_ALL)

    print(Fore.RED + f'Cardinality = {cardianliy}' + Style.RESET_ALL)

    return cardianliy, clean_expected_types

## Main Script 
### Convert the computationalTool Profile from a JSON to YAML Format
###### Prepare the YAML data, save then display it

In [ ]:
#Download the JSON-LD file
#!wget https://github.com/BioSchemas/specifications/blob/master/ComputationalTool/jsonld/ComputationalTool_v0.5-DRAFT.json 
in_file = "ComputationalTool_v0.5-DRAFT.json"

In [ ]:
with open(in_file, "r", encoding="utf-8") as i:
    data = json.load(i)

In [ ]:
#Display bioschemas:ComputationalTool Graph 
print(json.dumps(data['@graph'][0], indent=True))

In [ ]:
#For each profile : 
for g in data["@graph"]:
    
    print(Fore.BLUE + Style.BRIGHT + f'Profile : {g["@id"]}' + Style.RESET_ALL)     

    #Prepare the transfermed profile : spec_info & mapping fields
    transformed_profile = generate_transformed_profile(g)
    
    #Display the YAML
    #print(yaml.dump(transformed_profile, indent=4, default_flow_style=False))
    
    out_file = "generated_"+g["@id"].split(':')[1]+".yaml"
    
    with open(out_file, "w", encoding="utf-8") as o:
        yaml.dump(transformed_profile, o)
    
    #To display only the bioschemas:ComputationalTool
    break

In [ ]:
!ls

In [ ]:
!cat generated_computationalTool.yaml

## Inject the YAML in a HTML File
Ps: The profile's version will be the name of the profile HTML File

In [ ]:
# Note: The folder should be in the transformed_profile["spec_info"]["title"] folder

fileName= transformed_profile["spec_info"]["version"] +".html"
print(fileName)

top_of_the_page='''
---
redirect_from:
- "devSpecs/Tool/specification"
- "devSpecs/Tool/specification/"
- "/devSpecs/Tool/"
- "/devSpecs/Tool"
- "/specifications/drafts/Tool"
- "/specifications/drafts/Tool/"
- "/profiles/Tool/"
- "/profiles/Tool"
- "/profiles/ComputationalTool/"
- "/profiles/ComputationalTool"
- "/profiles/Tool/0.6-DRAFT/"

hierarchy:
- Thing
- CreativeWork
- SoftwareApplication

name: ComputationalTool

previous_version: 0.5-DRAFT
previous_release:

status: revision
spec_type: Profile
group: tools
use_cases_url: '/useCases/ComputationalTool'
cross_walk_url: https://docs.google.com/spreadsheets/d/12W7DQkUfsY0lrHEVvowgHXAcO2WJyNI6c8ZJzXgzoRI/edit
gh_tasks: https://github.com/Bioschemas/bioschemas/labels/type%3A%20Tool
live_deploy: /liveDeploys

parent_type: SoftwareApplication
hierarchy:
- Thing
- CreativeWork
- SoftwareApplication

# spec_info content generated using GOWeb
# DO NOT MANUALLY EDIT THE CONTENT
'''

yaml_delimiter="---"

with open(fileName, "w", encoding="utf-8") as o:
    o.write(top_of_the_page)
    yaml.dump(transformed_profile, o)
    o.write(yaml_delimiter)


In [1]:
!cat 0.5-DRAFT.html


---
redirect_from:
- "devSpecs/Tool/specification"
- "devSpecs/Tool/specification/"
- "/devSpecs/Tool/"
- "/devSpecs/Tool"
- "/specifications/drafts/Tool"
- "/specifications/drafts/Tool/"
- "/profiles/Tool/"
- "/profiles/Tool"
- "/profiles/ComputationalTool/"
- "/profiles/ComputationalTool"
- "/profiles/Tool/0.6-DRAFT/"

hierarchy:
- Thing
- CreativeWork
- SoftwareApplication

name: ComputationalTool

previous_version: 0.5-DRAFT
previous_release:

status: revision
spec_type: Profile
group: tools
use_cases_url: '/useCases/ComputationalTool'
cross_walk_url: https://docs.google.com/spreadsheets/d/12W7DQkUfsY0lrHEVvowgHXAcO2WJyNI6c8ZJzXgzoRI/edit
gh_tasks: https://github.com/Bioschemas/bioschemas/labels/type%3A%20Tool
live_deploy: /liveDeploys

parent_type: SoftwareApplication
hierarchy:
- Thing
- CreativeWork
- SoftwareApplication

# spec_info content generated using GOWeb
# DO NOT MANUALLY EDIT THE CONTENT
mapping:
- bsc_description: ''
  cardinality: ONE
  controlled_vocab: ''
  descri